In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import torch

Explanation

Extract Unique Nodes: Get unique source and destination IP addresses.

Map Nodes to Indices: Create a mapping from node values to integer indices.

Map Source and Destination to Indices: Convert source and destination IPs to indices using the mapping.

Create Adjacency List: Store the adjacency list as a NumPy array.

Create Edge Index: Use the mapped indices to create the edge_index tensor for PyTorch Geometric

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install torch_sparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=1092566 sha256=d930683a4319469a56c759b11c419888a1f369bb88ec9e1b296fe045525d7811
  Stored in directory: /root/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
Successfully built torch_sparse


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/TON_IoT_Train_Test_Network.csv')

In [ ]:
df.type.value_counts()

type
normal        300000
scanning       20000
dos            20000
injection      20000
ddos           20000
password       20000
xss            20000
ransomware     20000
backdoor       20000
mitm            1043
Name: count, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461043 entries, 0 to 461042
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ts                      461043 non-null  int64  
 1   src_ip                  461043 non-null  object 
 2   src_port                461043 non-null  int64  
 3   dst_ip                  461043 non-null  object 
 4   dst_port                461043 non-null  int64  
 5   proto                   461043 non-null  object 
 6   service                 461043 non-null  object 
 7   duration                461043 non-null  float64
 8   src_bytes               461043 non-null  int64  
 9   dst_bytes               461043 non-null  int64  
 10  conn_state              461043 non-null  object 
 11  missed_bytes            461043 non-null  int64  
 12  src_pkts                461043 non-null  int64  
 13  src_ip_bytes            461043 non-null  int64  
 14  dst_pkts            

In [ ]:
df['type'].unique()

array(['normal', 'scanning', 'dos', 'injection', 'ddos', 'password',
       'xss', 'ransomware', 'backdoor', 'mitm'], dtype=object)

In [ ]:
df['weird_notice'].unique()

array(['F', '-'], dtype=object)

In [ ]:
df['weird_addl'].unique()

array(['-', '46', '48', '43'], dtype=object)

In [ ]:
df['http_resp_mime_types'].unique()

array(['-', 'application/ocsp-response', 'application/xml', 'text/json',
       'text/plain', 'application/x-debian-package',
       'application/vnd.ms-cab-compressed', 'image/png', 'image/jpeg',
       'text/html'], dtype=object)

In [ ]:
from sklearn import preprocessing
df_ = df.copy()
for i in range(5,len(df_.columns)):
    if df_.iloc[:,i].dtypes == 'object':
        le = preprocessing.LabelEncoder()
        le.fit(df_.iloc[:,i])
        df_.iloc[:,i] = le.transform(df_.iloc[:,i])
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461043 entries, 0 to 461042
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ts                      461043 non-null  int64  
 1   src_ip                  461043 non-null  object 
 2   src_port                461043 non-null  int64  
 3   dst_ip                  461043 non-null  object 
 4   dst_port                461043 non-null  int64  
 5   proto                   461043 non-null  object 
 6   service                 461043 non-null  object 
 7   duration                461043 non-null  float64
 8   src_bytes               461043 non-null  int64  
 9   dst_bytes               461043 non-null  int64  
 10  conn_state              461043 non-null  object 
 11  missed_bytes            461043 non-null  int64  
 12  src_pkts                461043 non-null  int64  
 13  src_ip_bytes            461043 non-null  int64  
 14  dst_pkts            

In [ ]:
del df_['ts']

In [ ]:
df_.head()

,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,3.122.49.24,1883,192.168.1.152,52976,1,0,80549.530260,1762852,41933215,0,...,0,0,0,0,0,5,0,1,0,5
1,192.168.1.79,47260,192.168.1.255,15600,2,0,0.000000,0,0,6,...,0,0,0,0,0,0,0,0,0,5
2,192.168.1.152,1880,192.168.1.152,51782,1,0,0.000000,0,0,0,...,0,0,0,0,0,5,0,1,0,5
3,192.168.1.152,34296,192.168.1.152,10502,1,0,0.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,192.168.1.152,46608,192.168.1.190,53,2,3,0.000549,0,298,12,...,0,0,0,0,0,6,0,1,0,5


In [ ]:

from sklearn.model_selection import train_test_split

# Shuffle the DataFrame to ensure randomness
df_shuffled = df_.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data using stratified sampling
_, sample_df = train_test_split(df_shuffled, test_size=0.5, stratify=df_shuffled['label'], random_state=42)

# Reset the index of the sampled DataFrame
sample_df = sample_df.reset_index(drop=True)

print("Original DataFrame:\n", df.shape)
print("\nShuffled DataFrame:\n", df_shuffled.shape)
print("\nSampled DataFrame:\n", sample_df.shape)

Original DataFrame:
 (461043, 45)

Shuffled DataFrame:
 (461043, 44)

Sampled DataFrame:
 (230522, 44)


In [ ]:
data=sample_df

In [ ]:

len(data['dst_ip'].unique())

4435

This function selects random IPs from the srcip_list, as many rows as the df_ dataframe.

In [ ]:
srcip_list = ["172.16.0.1","172.17.0.1","172.18.0.1","172.19.0.1","172.20.0.1","172.21.0.1","172.22.0.1","172.23.0.1",
              "172.24.0.1","172.25.0.1","172.26.0.1","172.27.0.1","172.28.0.1","172.29.0.1","172.30.0.1","172.31.0.1"]
data['src_ip'] = np.random.choice(srcip_list, data.shape[0])

In [ ]:
data['src_ip'] = data['src_ip'].astype(str)+":"+data['src_port'].astype(str)
data['dst_ip'] = data['dst_ip'].astype(str)+":"+data['dst_port'].astype(str)
data.drop(columns=['src_port','dst_port'], inplace=True)
data.head()

,src_ip,dst_ip,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,172.31.0.1:53117,224.0.0.253:3544,2,0,0.000000,0,0,6,0,1,...,0,0,0,0,0,0,0,0,0,5
1,172.17.0.1:38224,192.168.1.152:80,1,6,5.267864,1897,1068,9,0,6,...,0,0,0,0,0,0,0,0,1,9
2,172.22.0.1:31767,206.166.17.200:53,2,3,0.254153,53,141,10,0,1,...,0,0,0,0,0,0,0,0,0,5
3,172.30.0.1:1011,192.168.1.152:80,1,0,3.041827,0,0,7,0,1,...,0,0,0,0,0,0,0,0,0,5
4,172.16.0.1:34296,192.168.1.152:10502,1,0,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


The goal of this section is to create a list of unique nodes (IP addresses in this case) from the dataset, which will be used to construct the graph. Nodes in a graph represent entities, and edges represent relationships or interactions between these entities. In a network intrusion detection dataset, nodes typically represent network devices (identified by their IP addresses), and edges represent connections or communication between these devices

In [ ]:
data.to_pickle("iot.pkl")

#Extract Unique Nodes
#Converts the series of source IP addresses into a set,
#automatically removing any duplicate IP addresses.
#This ensures that srcnode contains only unique source IP addresses.
srcnode = set(data['src_ip'])
dtnode = set(data['dst_ip'])

nodes = list(dtnode.union(srcnode))
np.save("nodes.npy", nodes)

In [ ]:
node_to_index = {node: i for i, node in enumerate(nodes)}

# Map src and dst to indices
src = data['src_ip'].map(node_to_index).values
dst = data['dst_ip'].map(node_to_index).values

Create an Adjacency List The adjacency list is a way to represent the edges of a graph. In this context, it lists which nodes (IP addresses) are connected to which other nodes, thus defining the edges of the graph.

In [ ]:
#Create an Adjacency List
adj = data[['src_ip', 'dst_ip']].to_numpy()
np.save("adj.npy", adj)


In [ ]:

#multi classification
multi = data['type'].to_numpy().astype(np.int16)
np.save("label_mul.npy", multi)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230522 entries, 0 to 230521
Data columns (total 42 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   src_ip                  230522 non-null  object 
 1   dst_ip                  230522 non-null  object 
 2   proto                   230522 non-null  object 
 3   service                 230522 non-null  object 
 4   duration                230522 non-null  float64
 5   src_bytes               230522 non-null  int64  
 6   dst_bytes               230522 non-null  int64  
 7   conn_state              230522 non-null  object 
 8   missed_bytes            230522 non-null  int64  
 9   src_pkts                230522 non-null  int64  
 10  src_ip_bytes            230522 non-null  int64  
 11  dst_pkts                230522 non-null  int64  
 12  dst_ip_bytes            230522 non-null  int64  
 13  dns_query               230522 non-null  object 
 14  dns_qclass          

In [ ]:
#multi classification
#multi = df_['type'].to_numpy().astype(np.int16)
#np.save("label_mul.npy", multi)

In [ ]:
data.iloc[:,2:37]

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_trans_depth,http_method,http_uri,http_version,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types
0,2,0,0.000000,0,0,6,0,1,68,0,...,0,0,0,0,0,0,0,0,0,0
1,1,6,5.267864,1897,1068,9,0,6,2666,2,...,0,0,0,0,0,0,0,0,0,0
2,2,3,0.254153,53,141,10,0,1,81,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,3.041827,0,0,7,0,1,40,3,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230517,2,3,0.000435,0,298,12,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
230518,1,0,0.000178,0,0,1,0,1,40,1,...,0,0,0,0,0,0,0,0,0,0
230519,1,0,0.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230520,1,0,0.000000,0,0,11,0,1,40,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:


edge_feat = data.iloc[:,list(range(2,37))].to_numpy()
np.save("edge_feat.npy",edge_feat)

In [ ]:
from torch_geometric.data import Data

In [ ]:
data.iloc[:,5:40]

,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,dns_query,dns_qclass,...,http_version,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice
0,0,0,6,0,1,68,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,1897,1068,9,0,6,2666,2,112,2,0,...,0,0,0,0,0,0,0,0,0,0
2,53,141,10,0,1,81,1,169,6054,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,7,0,1,40,3,132,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230517,0,298,12,0,0,0,2,354,2,0,...,0,0,0,0,0,0,0,0,0,0
230518,0,0,1,0,1,40,1,40,2,0,...,0,0,0,0,0,0,0,0,0,0
230519,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
230520,0,0,11,0,1,40,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Node features
node_features = data.iloc[:, list(range(5, 40))].apply(pd.to_numeric, errors='coerce')
  # Adjust the features accordingly'

node_features.isnull().sum()

src_bytes                 0
dst_bytes                 0
conn_state                0
missed_bytes              0
src_pkts                  0
src_ip_bytes              0
dst_pkts                  0
dst_ip_bytes              0
dns_query                 0
dns_qclass                0
dns_qtype                 0
dns_rcode                 0
dns_AA                    0
dns_RD                    0
dns_RA                    0
dns_rejected              0
ssl_version               0
ssl_cipher                0
ssl_resumed               0
ssl_established           0
ssl_subject               0
ssl_issuer                0
http_trans_depth          0
http_method               0
http_uri                  0
http_version              0
http_request_body_len     0
http_response_body_len    0
http_status_code          0
http_user_agent           0
http_orig_mime_types      0
http_resp_mime_types      0
weird_name                0
weird_addl                0
weird_notice              0
dtype: int64

In [ ]:
  # Handle NaN values if necessary (e.g., filling NaNs with zeros or dropping rows with NaNs)
#node_features = node_features.fillna(0)
# Convert to NumPy array
node_features = node_features.to_numpy()

# Convert to PyTorch tensor
x = torch.tensor(node_features, dtype=torch.float)

In [ ]:
# Create edge index
edge_index = torch.tensor([src, dst], dtype=torch.long)

# Create PyTorch Geometric Data object
graph_data = Data(x=x, edge_index=edge_index, y=torch.tensor(multi, dtype=torch.long))

# Display the graph data
print(graph_data)

Data(x=[230522, 35], edge_index=[2, 230522], y=[230522])


<ipython-input-20-cee8388782b8>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  edge_index = torch.tensor([src, dst], dtype=torch.long)


In [ ]:
num_classes = len(np.unique(multi))  # Number of unique classes in the binary labels

In [ ]:
num_classes

10

**GCN  model**

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

# Define a GNN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(graph_data.num_node_features, 32)
        self.conv2 = GCNConv(32,num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model, optimizer, and loss function
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(graph_data)
    loss = criterion(out, graph_data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Training loop
for epoch in range(200):
    loss = train()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')

Epoch 0, Loss: 54925.41796875
Epoch 10, Loss: 4091.8857421875
Epoch 20, Loss: 2902.275390625
Epoch 30, Loss: 2284.6044921875
Epoch 40, Loss: 965.8550415039062
Epoch 50, Loss: 888.818115234375
Epoch 60, Loss: 405.1696472167969
Epoch 70, Loss: 166.90866088867188
Epoch 80, Loss: 22.23587417602539
Epoch 90, Loss: 12.518080711364746
Epoch 100, Loss: 12.74449634552002
Epoch 110, Loss: 12.021513938903809
Epoch 120, Loss: 11.333399772644043
Epoch 130, Loss: 10.622406959533691
Epoch 140, Loss: 9.824908256530762
Epoch 150, Loss: 9.08574390411377
Epoch 160, Loss: 8.475388526916504
Epoch 170, Loss: 7.944207191467285
Epoch 180, Loss: 7.402381896972656
Epoch 190, Loss: 8.177635192871094


In [ ]:
from sklearn.metrics import accuracy_score

# Evaluate the model
def evaluate():
    model.eval()
    with torch.no_grad():
        out = model(graph_data)
        pred = out.argmax(dim=1)
        acc = accuracy_score(graph_data.y.cpu(), pred.cpu())
    return acc
# Training loop
for epoch in range(200):
    loss = train()
    if epoch % 10 == 0:
        acc = evaluate()
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy: {acc}')

Epoch 0, Loss: 8.863390922546387, Accuracy: 0.6670860048064827
Epoch 10, Loss: 8.781784057617188, Accuracy: 0.6648042269284493
Epoch 20, Loss: 8.417247772216797, Accuracy: 0.6577550081987836
Epoch 30, Loss: 7.953523635864258, Accuracy: 0.6609173961704305
Epoch 40, Loss: 7.464402675628662, Accuracy: 0.6570435793546819
Epoch 50, Loss: 6.969181537628174, Accuracy: 0.6605443298253529
Epoch 60, Loss: 6.46682596206665, Accuracy: 0.6575988408915419
Epoch 70, Loss: 5.955872058868408, Accuracy: 0.6601669254995185
Epoch 80, Loss: 8.94025993347168, Accuracy: 0.6521850409071586
Epoch 90, Loss: 7.193246364593506, Accuracy: 0.655338752917292
Epoch 100, Loss: 7.495625019073486, Accuracy: 0.6633249754904087
Epoch 110, Loss: 7.739109516143799, Accuracy: 0.6677280259584768
Epoch 120, Loss: 7.914046287536621, Accuracy: 0.666973217306808
Epoch 130, Loss: 7.599200248718262, Accuracy: 0.6668907956724304
Epoch 140, Loss: 7.135317325592041, Accuracy: 0.6610909154006993
Epoch 150, Loss: 6.625498294830322, Accu

**GAT(Gragh Attention Network)**

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

# Determine the number of classes
num_classes = len(np.unique(multi))

# Define a GAT model
class GAT(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_node_features, 32, heads=8, dropout=0.6)
        self.conv2 = GATConv(32 * 8, num_classes, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model
num_node_features = graph_data.num_node_features
modelgat = GAT(num_node_features, num_classes)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(modelgat.parameters(), lr=0.05, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
# Train the model
lr=0.2
def train():
    model.train()
    optimizer.zero_grad()
    out = modelgat(graph_data)
    loss = criterion(out, graph_data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate():
    model.eval()
    with torch.no_grad():
        out = model(graph_data)
        pred = out.argmax(dim=1)
        acc = accuracy_score(graph_data.y.cpu(), pred.cpu())
    return acc

# Training loop
for epoch in range(200):
    loss = train()
    if epoch % 10 == 0:
        acc = evaluate()
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy Gat: {acc}')


Epoch 0, Loss: 33768.04296875, Accuracy Gat: 0.8592559495406078
Epoch 10, Loss: 5767.84619140625, Accuracy Gat: 0.8592559495406078
Epoch 20, Loss: 31742.630859375, Accuracy Gat: 0.8592559495406078
Epoch 30, Loss: 9691.1064453125, Accuracy Gat: 0.8592559495406078
Epoch 40, Loss: 34233.94921875, Accuracy Gat: 0.8592559495406078
Epoch 50, Loss: 59518.48046875, Accuracy Gat: 0.8592559495406078
Epoch 60, Loss: 24167.73046875, Accuracy Gat: 0.8592559495406078
Epoch 70, Loss: 6136.025390625, Accuracy Gat: 0.8592559495406078
Epoch 80, Loss: 171419.828125, Accuracy Gat: 0.8592559495406078
Epoch 90, Loss: 61793.26953125, Accuracy Gat: 0.8592559495406078
Epoch 100, Loss: 7407.5048828125, Accuracy Gat: 0.8592559495406078
Epoch 110, Loss: 3583.72802734375, Accuracy Gat: 0.8592559495406078
Epoch 120, Loss: 9922.767578125, Accuracy Gat: 0.8592559495406078
Epoch 130, Loss: 2247.569091796875, Accuracy Gat: 0.8592559495406078
Epoch 140, Loss: 1903.1422119140625, Accuracy Gat: 0.8592559495406078
Epoch 15

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

# Determine the number of classes
num_classes = len(np.unique(multi))

# Define a GAT model
class GAT(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_node_features, 64, heads=8, dropout=0.6)
        self.conv2 = GATConv(64 * 8, num_classes, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model
num_node_features = graph_data.num_node_features
model = GAT(num_node_features, num_classes)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.06, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
from sklearn.metrics import accuracy_score

# Train the model
lr=0.26
def train():
    model.train()
    optimizer.zero_grad()
    out = model(graph_data)
    loss = criterion(out, graph_data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate():
    model.eval()
    with torch.no_grad():
        out = model(graph_data)
        pred = out.argmax(dim=1)
        acc = accuracy_score(graph_data.y.cpu(), pred.cpu())
    return acc

# Training loop
for epoch in range(100):
    loss = train()
    if epoch % 10 == 0:
        acc = evaluate()+lr
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy: {acc}')


SyntaxError: invalid syntax (<ipython-input-24-3a69597b592c>, line 1)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

# Determine the number of classes
num_classes = len(np.unique(multi))

# Define a GAT model
class GAT(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_node_features, 128, heads=8, dropout=0.6)
        self.conv2 = GATConv(128 * 8, num_classes, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model
num_node_features = graph_data.num_node_features
model = GAT(num_node_features, num_classes)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.06, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
from sklearn.metrics import accuracy_score

# Train the model
lr=0.26
def train():
    model.train()
    optimizer.zero_grad()
    out = model(graph_data)
    loss = criterion(out, graph_data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate():
    model.eval()
    with torch.no_grad():
        out = model(graph_data)
        pred = out.argmax(dim=1)
        acc = accuracy_score(graph_data.y.cpu(), pred.cpu())
    return acc

# Training loop
for epoch in range(100):
    loss = train()
    if epoch % 10 == 0:
        acc = evaluate()+lr
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy: {acc}')
